Let’s work on the following:
1. Access the CB API (with my login info), which is inside Dropbox. Download the list of companies founded in 2016 and with category ‘Artificial Intelligence’. That should be 500sh companies, we start with a small sample.
2. For each of those companies, download all the NEWS that Crunchbase has. Try to put those news into an easy format: maybe a TXT file per news, and all TXT files of a company into one folder. And remove all non-relevant components (like Ads or other text). This will make it easier if somebody in the team will have to do some reading.
3. Once this looks good, we’ll do some NLP on the text. But let’s get the "raw data" first.
4. Please upload all material on Dropbox.

In [ ]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import nltk
from nltk import *
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
import codecs
import string
import random
import re

import requests
import urllib
import bs4
from bs4 import BeautifulSoup
from readability.readability import Document # https://github.com/buriy/python-readability. Tried Goose, Newspaper (python libraries on Github). Bad results.
from http.cookiejar import CookieJar # 

import time
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import datetime
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.dates as mdates

import ast
from itertools import chain
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
import shutil

import articleDateExtractor
from newspaper import Article

import pycrunchbase
cb = pycrunchbase.CrunchBase('662e263576fe3e4ea5991edfbcfb9883')

In [ ]:
# df = pd.DataFrame(columns=["permalink"])

# url = 'https://api.crunchbase.com/v3.1/organizations/?user_key=662e263576fe3e4ea5991edfbcfb9883'
# # location given here 
# location = "philadephia"

# # defining a params dict for the parameters to be sent to the API 
# PARAMS = {'address':location} 

# while True:
#     try:
#         response = requests.get(url, PARAMS)
#         json = response.json()
#         for c in json['data']['items']:
#             permalink=c['properties']['permalink']
#             if(permalink!=None):
#                 categories_json = requests.get("https://api.crunchbase.com/v3.1/organizations/" 
#                                 + permalink+"/categories?user_key=662e263576fe3e4ea5991edfbcfb9883", PARAMS).json()
#                 for cat in categories_json['data']['items']:
#                     if(cat["properties"]["category_groups"]!=None):
#                         if("Artificial Intelligence" in cat["properties"]["category_groups"]):
#                             print(permalink)
#                             df = df.append({'permalink' : permalink} , ignore_index=True)
#         url = json['data']['paging']['next_page_url'] + "&user_key=662e263576fe3e4ea5991edfbcfb9883"
#         print(url)
#     except:
#         print("error: " + url)
        
# df

df = pd.read_csv('csv_export/organizations.csv')
print(df.columns)
df

In [ ]:
ai_df = pd.DataFrame(columns = df.columns)
for i in range(len(df)):
    try:
        if("Artificial Intelligence" in df['category_group_list'][i] and '2016' in df['founded_on'][i]):
            ai_df = ai_df.append(df.loc[i])
    except:
        pass
ai_df

In [ ]:
ai_df= ai_df.reset_index(drop=True)
ai_df

In [ ]:
# location given here 
location = "philadephia"

# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
for i in range(len(ai_df)):
    permalink = ai_df['permalink'][i].split("/")[2]
    newpath = '/Users/ksun/downloads/startup-data/ai-organizations/' + permalink
    if not os.path.exists(newpath):
        print("create new folder", newpath)
        os.makedirs(newpath)
    
        url = 'https://api.crunchbase.com/v3.1/organizations/' + permalink+'?relationships=news&user_key=662e263576fe3e4ea5991edfbcfb9883'
        response_json = requests.get(url, PARAMS).json()

        for n in response_json['data']['relationships']['news']['items']:
            try:
                uri=n['properties']['url']
                opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
                opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
                html =  opener.open(uri).read().decode('utf-8')
                readable_article = Document(html).summary()
                soup = BeautifulSoup(readable_article, "lxml")
                text = soup.get_text()

                name_of_file = uri.replace(".", "-").replace(":", "-").replace("/", "-")
                path = newpath + "/"

                completeName = os.path.join(path, name_of_file+".txt")  


                file1 = open(completeName, "w")

                file1.write(text)

                file1.close()
            except:
                pass
            
    if len(os.listdir(newpath)) == 0:
        print(newpath)
    
        url = 'https://api.crunchbase.com/v3.1/organizations/' + permalink+'?relationships=news&user_key=662e263576fe3e4ea5991edfbcfb9883'
        response_json = requests.get(url, PARAMS).json()
        print(url)

        for n in response_json['data']['relationships']['news']['items']:
            try:
                uri=n['properties']['url']
                print(uri)
                opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
                opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
                html =  opener.open(uri).read().decode('utf-8')
                readable_article = Document(html).summary()
                soup = BeautifulSoup(readable_article, "lxml")
                text = soup.get_text()

                name_of_file = uri.replace(".", "-").replace(":", "-").replace("/", "-")
                path = newpath + "/"

                completeName = os.path.join(path, name_of_file+".txt")  


                file1 = open(completeName, "w")

                file1.write(text)
                if(len(text) != 0):
                    print("write complete:", newpath)

                file1.close()
            except:
                pass


Hi Kevin
Thanks again for coming in! And great job – this is an excellent first step.
Here are the next steps:
1. I went through the folders – most look good, but it’s possible there’s some missing material. Spend a little more time to perfect the downloading  code. See if you can fix whichever errors you find through the process. It might be best to make this code as smooth as possible now, before we scale to working with lots of companies.
2. I am assuming the scraping process is OK, but it’s probably helpful to take one more look at that too (i.e. for example, those few lines missing in the TXT we looked at together may happen elsewhere too, no huge deal but worth looking into it).
3. Once you’ve finalized the downloading process, let’s start with a very simple exercise on the TXT files. For each TXT of each company, write a code that counts the number of times the following keywords are mentioned: PROTOTYPE, TEST, EXPERIMENT, MINIMUM VIABLE PRODUCT, BETA, ALPHA. This is just a start, we’ll do more of this, but let’s see how this goes. You can then input this data into an Excel file (one row per company).
 
If possible, please write your Python code clearly and upload it. So I can try to take a look.
 
Please remember to submit your timesheet by Thursday night each week. Let’s try to meet week after Spring break, I am very flexible. Of course, please reach out if you have any questions at any point.
Thanks so much!
A

In [ ]:
rootdir = '/Users/ksun/downloads/startup-data/ai-organizations/'

ai_df['articles'] = np.empty((len(ai_df), 0)).tolist()

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if '.DS_Store' not in file:
#             if(len(ai_df[ai_df['permalink'].astype(str).str.contains('/organization/' + subdir.split("/")[-1])])==0):
#                 print('/organization/' + subdir.split("/")[-1])
#                 shutil.rmtree(subdir)
#                 break
            row_index = ai_df.loc[ai_df['permalink'] == '/organization/' + subdir.split("/")[-1]].index.values.astype(int)[0]
            file = open(subdir +  "/" + file, "r")
            ai_df['articles'][row_index].append(file.read())
            file.close()

In [ ]:
# ai_df.to_csv('csv_export/ai_df.csv', index=False)

In [ ]:
# ai_df = pd.read_csv('csv_export/ai_df_old.csv', index_col=0)
# ai_df
# ai_df.drop(['Unnamed: 0'], axis=1)
# ai_df.to_csv('csv_export/ai_df.csv', index=False)

1. Gather good data from twitter (make sure to scrape over several months, stop at December 2018
2. topic clusters, sentiment, OVER TIME (month level) for a specific company
3. Take one old company like Google and track over time
__________________________
Let’s continue to work on the 2016 sample of AI companies for now, just to keep it simple.
1. Let’s add news from TWITTER and GOOGLE NEWS. You decide how to better get the right material from these two sources.
2. Once we have all these sources in place, we can start by doing a simple sentiment analysis. Each company should have a time series of sentiment measures (i.e. number of negative news): so each company should be have observations between first month of life and Dec 2018.

In [ ]:
# location given here 
location = "philadephia"

ai_df['articles'] = np.empty((len(ai_df), 0)).tolist()

# defining a params dict for the parameters to be sent to the API 
HEADERS = {'accept':"application/json"} 
for i in range(len(ai_df)):
    permalink = ai_df['permalink'][i].split("/")[2]
    
    url = 'https://api.crunchbase.com/v3.1/organizations/' + permalink+'?relationships=news&user_key=662e263576fe3e4ea5991edfbcfb9883'
    try:
        response_json = requests.get(url, headers=HEADERS, params={"$filter":"Path eq '/xxxxxx/'"}).json()
        print(url)

        for n in response_json['data']['relationships']['news']['items']:

            try:
                uri=n['properties']['url']
                opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
                opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
                html =  opener.open(uri).read().decode('utf-8')
                readable_article = Document(html).summary()
                soup = BeautifulSoup(readable_article, "lxml")
                text = soup.get_text()

    #             article = Article(uri)
    #             article.parse()
                date = n['properties']['posted_on']
    #             date = articleDateExtractor.extractArticlePublishedDate(uri)


                ai_df['articles'][i].append([uri, date, text])


            except:
                print("error:", permalink)
    except:
        print("error", url)
        #ugly-research
        #recomomenderx, geopipe-inc, cloudzero, ekkono-solutions, trials-ai, quali-fit, vidrovr, ar-cadia, ns8-inc, vapispace, satisfy, gravyty, ignite-biosciences

In [ ]:
count = 0
for i in range(len(ai_df)):
    if(len(ai_df['articles'][i])>10):
        count+=1
print(count)

Thanks for coming in, Kevin!
 
I think we’re doing great progress.
 
Here's the next steps of this  task:
1. Let’s focus on just 1 company and just Twitter. We should try to get all tweets from that company and about that company (from the very beginning to Dec 31st 2018).
2. Let’s allocate the tweets on each month from the month in which the Twitter account opens.
3. Let’s run sentiment analysis and graph sentiment over all months. You could probably do a separate graph for tweets from the company and tweets about the company. The graph should be a time series, where axis X is date and Y is sentiment. It should basically look like a curve that goes on over time.
4. To do this, pick a good company: one that has enough tweets.
5. If the process is not to heavy, let’s scale it to 100 companies. Just pick 100 companies from your sample.
 
You can take a look at how much it’d be to move the process to Amazon Web Services. Let me know how it looks and we can then decide.
 
If possible, please send me an update sometimes next week and then we can probably meet the week of Apr 8th. (No need to meet next week!)
 
Thanks again!
A

In [ ]:
ai_df['twitter_url'][142].split("/")[-2]

In [ ]:
#  python Exporter.py --username "viralheat" --since 2016-01-01 --until 2018-12-31
# python Exporter.py --username "kernelco" --since 2016-01-01 --until 2018-12-31 --maxtweets 100
# python main.py --username "kernelco" --since 2016-01-01 --until 2018-12-31 --max-tweets 1000000

In [ ]:
ai_df['num_prototype'] = 0
ai_df['num_test'] = 0
ai_df['num_experiment'] = 0
ai_df['num_minimum_viable_product'] = 0
ai_df['num_beta'] = 0
ai_df['num_alpha'] = 0
for i in range(len(ai_df)):
    wordlist = []
    for a in ai_df['articles'][i]:
        wordlist.extend(list(chain(*[word_tokenize(s) for s in sent_tokenize(a)])))
    counter = Counter(wordlist)
    ai_df.loc[i, 'num_prototype'] = counter['prototype']
    ai_df.loc[i, 'num_test'] = counter['test']
    ai_df.loc[i, 'num_experiment'] = counter['experiment']
    ai_df.loc[i, 'num_minimum_viable_product'] = counter['viable']
    ai_df.loc[i, 'num_beta'] = counter['beta']
    ai_df.loc[i, 'num_alpha'] = counter['alpha']

In [ ]:
rootdir = '/Users/ksun/downloads/startup-data/GetOldTweets-python'

ai_df['tweets'] = np.empty((len(ai_df), 0)).tolist()

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if '.DS_Store' not in file and file in ['mobalytics-2.csv', 'swisscognitive.csv', 'prisma-5.csv']:
#             if(len(ai_df[ai_df['permalink'].astype(str).str.contains('/organization/' + subdir.split("/")[-1])])==0):
#                 print('/organization/' + subdir.split("/")[-1])
#                 shutil.rmtree(subdir)
#                 break
            row_index = ai_df.loc[ai_df['permalink'] == '/organization/' + file[0:-4]].index.values.astype(int)[0]
#             file = open(subdir +  "/" + file, "r")
            print(subdir +  "/" + file)
            temp_tweet_df = pd.read_csv(subdir +  "/" + file, error_bad_lines=False, sep=';')
            for j in range(len(temp_tweet_df)):
                try:
                    ai_df['tweets'][row_index].append([str(temp_tweet_df['username'][j]),
                                                      temp_tweet_df['date'][j],
                                                      temp_tweet_df['retweets'][j],
                                                      temp_tweet_df['favorites'][j],
                                                      str(temp_tweet_df['text'][j])])
                except:
                    continue
                    
# import shutil
# rootdir = '/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets'

# ai_df['tweets'] = np.empty((len(ai_df), 0)).tolist()

# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         if '.DS_Store' not in file:
# #             if(len(ai_df[ai_df['permalink'].astype(str).str.contains('/organization/' + subdir.split("/")[-1])])==0):
# #                 print('/organization/' + subdir.split("/")[-1])
# #                 shutil.rmtree(subdir)
# #                 break
#             row_index = ai_df.loc[ai_df['permalink'] == '/organization/' + file.split("/")[-1][0:-4]].index.values.astype(int)[0]
# #             file = open(subdir +  "/" + file, "r")
#             print(subdir +  "/" + file)
#             temp_tweet_df = pd.read_csv(subdir +  "/" + file, error_bad_lines=False)
#             for j in range(len(temp_tweet_df)):
#                 try:
#                     ai_df['tweets'][row_index].append([str(temp_tweet_df['username'][j]),
#                                                       str(temp_tweet_df['user_handle'][j]),
#                                                       temp_tweet_df['date'][j],
#                                                       temp_tweet_df['retweets'][j],
#                                                       temp_tweet_df['favorites'][j],
#                                                       str(temp_tweet_df['text'][j])])
#                 except:
#                     continue

In [ ]:
# ai_df.to_csv('csv_export/ai_df.csv', index=False)
ai_df = pd.read_csv('csv_export/ai_df.csv')
ai_df
# ai_df.drop(['Unnamed: 0'], axis=1)
# ai_df.to_csv('csv_export/ai_df.csv', index=False)

In [ ]:
def preprocessTweet(tweet): # preprocess tweets
    tweet = re.sub(r'\d+', '', str(tweet)) # remove numbers
    tweet = tweet.lower() # convert to lower case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # convert www.* or https?://* to URL
    tweet = re.sub('@[^\s]+','AT_USER',tweet) # convert @username to AT_USER
    tweet = re.sub('[\s]+', ' ', tweet) # remove additional white spaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # replace #word (hashtags) with word
    tweet = tweet.strip('\'"') # trim
    return tweet    

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

preprocessTweet("@VirginAmerica @virginmedia I'm flying your #fabulous #Seductive skies again! U take all the #stress away from travel http://t.co/ahlXHhKiyn")

In [ ]:
def getStopWordList(stopWordListFileName): # get stopword list
    sw = [] # create list of stopwords
    
    nltk_stopwords = stopwords.words('english')
    for w in nltk_stopwords:
        sw.append(w)
    sw.append('AT_USER') # special stopwords from preprocessTweet function
    sw.append('URL')
    
    fp = open(stopWordListFileName, 'r') # load in any more custom stopwords from file
    line = fp.readline()
    while line:
        word = line.strip()
        sw.append(word)
        line = fp.readline()
    fp.close()
    return sw


stopword_list = getStopWordList('stopwords.txt')
print('Stopword List: ', stopword_list)

In [ ]:
def getFeatureVector(tweet): 
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        w = replaceTwoOrMore(w) # replace two or more with two occurrences
        w = w.strip('\'"?,.') # strip punctuation
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w) # check if the word starts with an alphabet
        if(w in stopword_list or val is None): # ignore if it is a stop word
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
 
getFeatureVector(preprocessTweet("@VirginAmerica @virginmedia I'm flying your #fabulous #Seductive skies again! U take all the #stress away from travel http://t.co/ahlXHhKiyn"))

In [ ]:
airline_train = pd.read_csv("Twitter-Get-Old-Tweets-Scraper/training.1600000.processed.noemoticon.csv", 
                          header=None, encoding="ISO-8859-1") #latin1 encoding
airline_train.columns = ["sentiment", "id", "date", "query", "user", "text"]
# airline_train = airline_train.sample(frac=0.05, replace=True, random_state=1)
# airline_train = airline_train.reset_index(drop=True)
airline_tweets = []
airline_featurelist = []
for i in range(len(airline_train)):
    sentiment = airline_train['sentiment'][i]
    tweet = airline_train['text'][i]
    preprocessedTweet = preprocessTweet(tweet)
    featureVector = getFeatureVector(preprocessedTweet)
    airline_featurelist.extend(featureVector)
    airline_tweets.append((featureVector, sentiment))
        
def extract_features(tweet): # Determine if tweet contains a feature word
    tweet_words = set(tweet)
    features = {}
    for word in airline_featurelist:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

airline_featurelist = list(set(airline_featurelist)) # remove airline_featurelist duplicates
airline_featurelist

In [ ]:
airline_training_set = nltk.classify.util.apply_features(extract_features, airline_tweets)
airline_NBClassifier = nltk.NaiveBayesClassifier.train(airline_training_set)

In [ ]:
from sklearn.externals import joblib
# joblib.dump(airline_NBClassifier, 'csv_export/NBClassifier.pkl') 
# NBClassifier = joblib.load('csv_export/NBClassifier.pkl') 

In [ ]:
# https://www.kaggle.com/paoloripamonti/twitter-sentiment-analysis
model = load_model('twitter-sentiment-model/model.h5')
tokenizer = joblib.load('twitter-sentiment-model/tokenizer.pkl') 

In [ ]:
# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.5, 0.5)

def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score < SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [ ]:
print(ai_df.loc[ai_df['permalink'] == '/organization/' + 'prisma-5'].index.values.astype(int)[0])
print(ai_df.loc[ai_df['permalink'] == '/organization/' + 'swisscognitive'].index.values.astype(int)[0])
print(ai_df.loc[ai_df['permalink'] == '/organization/' + 'mobalytics-2'].index.values.astype(int)[0])

In [ ]:
ast.literal_eval(str(ai_df['tweets'][814]))

In [ ]:
# for i in range(len(ai_df)):
#     tweets = ast.literal_eval(ai_df['tweets'][1904])
#     for t in tweets:
#         t.append(NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet(t[5])))))
#     ai_df.loc[i, 'tweets'] = str(tweets)

tweets = ast.literal_eval(str(ai_df['tweets'][67]))
for t in tweets:
#     try:
#         del t[6]
#     except:
#         pass
    s = predict(preprocessTweet(t[4]))['label']
    t.append(s)
ai_df.loc[67, 'tweets'] = str(tweets)
tweets = ast.literal_eval(str(ai_df['tweets'][1893]))
for t in tweets:
#     try:
#         del t[6]
#     except:
#         pass
    s = predict(preprocessTweet(t[4]))['label']
    t.append(s)
ai_df.loc[1893, 'tweets'] = str(tweets)
tweets = ast.literal_eval(str(ai_df['tweets'][814]))
for t in tweets:
#     try:
#         del t[6]
#     except:
#         pass
    s = predict(preprocessTweet(t[4]))['label']
    t.append(s)
ai_df.loc[814, 'tweets'] = str(tweets)
ai_df

In [ ]:
ai_df_vis = pd.DataFrame(columns=['permalink', 'day', 'negative_count', 'positive_count'])

checked_days = [] # keep track of which days we have already added to airline_data
for i in range(len(ai_df)):
    if i==814:
        tweets = ast.literal_eval(ai_df['tweets'][i])
        for j in range(len(tweets)):
            cur_date = str(datetime.datetime.strptime(tweets[j][1].split(" ")[0], '%Y-%m-%d').strftime('%Y-%m'))
            if cur_date not in checked_days:
                print(tweets[j][1])
                checked_days.append(cur_date)
                try:
                    temp = list(filter(lambda x: str(datetime.datetime.strptime(x[1].split(" ")[0], '%Y-%m-%d').strftime('%Y-%m')) == cur_date and x[0]== 'MobalyticsHQ', tweets))
                    print(temp)

                    neg = 0 # negative count
                    pos = 0 # positive count
                    for k in range(len(temp)):
                        if temp[k][5] == 'NEGATIVE':
                            neg += 1
                        elif temp[k][5] == 'POSITIVE':
                            pos += 1
                    temp_df = pd.DataFrame([[ai_df.loc[i, 'permalink'], cur_date, neg, pos]], columns=['permalink', 'day', 'negative_count', 'positive_count'])
                    print(temp_df) # keep track of progress
                    ai_df_vis = ai_df_vis.append(temp_df, ignore_index=True)
                except:
                    print("error", cur_date)

In [ ]:
ai_df_vis.day = pd.to_datetime(ai_df_vis['day'], format='%Y-%m')
ai_df_vis.plot(x='day', y=['negative_count', 'positive_count'], figsize=(15,5), grid=True, color=['r', 'g'])

In [ ]:
ai_df_vis

In [ ]:
ax = ai_df_vis.plot(x='day', y=['negative_count', 'positive_count'], figsize=(50,5), grid=True, x_compat=True)
dates.datestr2num(date)
# set monthly locator
ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
# set formatter
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
# set font and rotation for date tick labels
plt.gcf().autofmt_xdate()

In [ ]:
# only Google news api only 6 months ago, very hard to get old news articles
from GoogleNews import GoogleNews
googlenews = GoogleNews()
googlenews.search('yunxi-technology')
googlenews.getpage(2)
googlenews.result()
ai_df['google_articles'] = np.empty((len(ai_df), 0)).tolist()